In [2]:
import os

from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import tensorflow._api.v2.compat.v1 as tf
import sklearn
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut

from models import deeper_fcn as architecture
import algorithms.heartrate as hr
import utils

2023-03-06 00:53:15.938550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 00:53:16.994734: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-06 00:53:16.994830: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-06 00:53:16.994841: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# tensorflow settings
tf.logging.set_verbosity(tf.logging.ERROR)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

In [6]:
# load data
# x_data_train, y_data_train, groups_train = ...
# dummy:
modelpath = "output"
train_size = 1000
n_groups = 28
data = np.genfromtxt('data/experiments/sleeprawlive.csv', delimiter=',')
# print the shape of the data
print(data.shape)
data = data[:len(data)-len(data)%400]
# reshape the data to be 2D array with 400 columns and 5 rows   (400, 5)
data = data.reshape( int(len(data)/400), 400)
# print the shape of the data
print(data.shape)
x_data_train = np.expand_dims(data, axis=2)
#y_train section
data = np.genfromtxt('data/experiments/gold.csv', delimiter=',')
print(len(data))
# compute average each 8 elements in the array
data = data[:len(data)-len(data)%16]
y_data_train = np.mean(data.reshape(int(len(data)/16), 16), axis=1)
print(y_data_train.shape)
# print first 2 rows of the data
print(y_data_train[:2])

groups_train = np.sort(np.random.randint(n_groups, size=train_size))

print(x_data_train.shape, y_data_train.shape, groups_train.shape)

#y_data_train = y_data_train.values
#x_data_train= x_data_train.values



2353324
(147082,)
[28.4375 17.5625]
(2938, 400, 1) (147082,) (1000,)


"\n    # read xtrain data from csv of bcg and store in numpy array\n    data = np.genfromtxt('data/mr1.csv', delimiter=',')\n    # print the shape of the data\n    print(data.shape)\n    data = data[:len(data)-len(data)%400]\n    # reshape the data to be 2D array with 400 columns and 5 rows   (400, 5)\n    data = data.reshape( int(len(data)/400), 400)\n    # print the shape of the data\n    print(data.shape)\n    x_train = np.expand_dims(data, axis=2)\n    data = np.genfromtxt('data/experiments/gold.csv', delimiter=',')\n    print(len(data))\n    # compute average each 8 elements in the array\n    data = data[:len(data)-len(data)%16]\n    y_train = np.mean(data.reshapvscode-cpptoolse(int(len(data)/16), 16), axis=1)\n    print(y_train.shape)\n    # print first 2 rows of the data\n    print(y_train[:2])\n"

(1175346,)
(2938, 400)
2353324
(147082,)
[28.4375 17.5625]
(2938, 400, 1) (147082,) (1000,)


"\n    # read xtrain data from csv of bcg and store in numpy array\n    data = np.genfromtxt('data/mr1.csv', delimiter=',')\n    # print the shape of the data\n    print(data.shape)\n    data = data[:len(data)-len(data)%400]\n    # reshape the data to be 2D array with 400 columns and 5 rows   (400, 5)\n    data = data.reshape( int(len(data)/400), 400)\n    # print the shape of the data\n    print(data.shape)\n    x_train = np.expand_dims(data, axis=2)\n    data = np.genfromtxt('data/experiments/gold.csv', delimiter=',')\n    print(len(data))\n    # compute average each 8 elements in the array\n    data = data[:len(data)-len(data)%16]\n    y_train = np.mean(data.reshapvscode-cpptoolse(int(len(data)/16), 16), axis=1)\n    print(y_train.shape)\n    # print first 2 rows of the data\n    print(y_train[:2])\n"

In [1]:
enlarge = 1
model_params = dict(metrics=["mae", "mape"], enlarge=enlarge)
fit_params = dict(epochs=30, verbose=2)  # set epochs between 30 and 75

modelname = (architecture.__name__ + "-x{}".format(enlarge))
modelpath = os.path.join("output", modelname)
os.makedirs(os.path.join(modelpath, "final"), exist_ok=True)
print(modelpath)

# write model architecture to JSON file
model = architecture.create(**model_params)
with open(os.path.join(modelpath, "model.json"), "w") as fp:
    fp.write(model.to_json())

NameError: name 'architecture' is not defined

In [ ]:
def process_split(xt, yt, i, fit_params):
    # set allow_growth in subprocess
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.keras.backend.set_session(tf.Session(config=config))
    
    csv_path = os.path.join(modelpath, "logs-{:02d}.csv".format(i))
    weights_path = os.path.join(modelpath, "weights-{:02d}.h5".format(i))
    fit_params.update(callbacks=[tf.keras.callbacks.CSVLogger(csv_path)])
    
    model = architecture.create(**model_params)
    r = model.fit(xt, yt, **fit_params)
    
    model.save_weights(weights_path)
    tf.keras.backend.clear_session()

    return r.history

In [8]:
# split training data with LeaveOneGroupOut cross validation
splitter = utils.get_group_splitter(n_groups, groups_train)

In [9]:
model = utils.get_model_from_json(modelpath, "model.json")

# calculate MAPE and MAE for each left-out patient
splitter = LeaveOneGroupOut().split(x_data_train, y_data_train, groups=groups_train)
results = []
for i, (t_inds, v_inds) in enumerate(splitter):
    model.load_weights(os.path.join(modelpath, "weights-{:02d}.h5".format(i)))
    y_pred = model.predict(x_data_train[v_inds])
    y_true = y_data_train[v_inds]
    results.append((hr.hr_mape(y_true, y_pred), hr.hr_mae(y_true, y_pred)))
results = np.array(results)
print(results)

FileNotFoundError: [Errno 2] No such file or directory: 'output/model.json'

In [ ]:
# train one model on entire training set

model = architecture.create(**model_params)
r = model.fit(x_data_train, y_data_train, **fit_params)
model.save_weights(os.path.join(modelpath, "final", "weights-00.h5"))
tf.keras.backend.clear_session()